In [1]:
import numpy as np 
import pandas as pd 
import sklearn
import warnings

In [2]:
def warn(*args, **kwargs):
    pass 
warnings.warn = warn

In [3]:
# file csv yang akan diproses
file_1 = 'gas.csv'

# panggil  filenya dan masukan ke dalam variabel data_n
data_1 = pd.read_csv(file_1)

# lihat dimensi semua filenya 
print('(n_baris, n_kolom)')
print('data_1 :\n', data_1.shape)

(n_baris, n_kolom)
data_1 :
 (12, 5)


### DATA_1

In [4]:
data_1

,Gas,Slongsong Gas,Tali Gas,Oli,Hasil
0,Normal,Bagus,Bagus,<1bulan,Tidak ada yang diganti
1,Normal,Bagus,Bagus,>1bulan,Oli diganti
2,Normal,Bagus,Putus,>1bulan,Tali gas diganti kemudian oli
3,Normal,Berkarat,Putus,>1bulan,Slongsong gas diganti kemudian tali gas kemudi...
4,Berat,Berkarat,Putus,>1bulan,Semua komponen diganti berurutan
5,Normal,Bagus,Putus,<1bulan,Tali gas diganti
6,Normal,Berkarat,Bagus,<1bulan,Slongsong gas diganti
7,Berat,Bagus,Bagus,<1bulan,Butuh pengecekan lebih lanjut
8,Berat,Berkarat,Putus,<1bulan,Slongsong gas diganti kemudian tali gas kemudi...
9,Normal,Bagus,Bagus,>1bulan,Tidak ada yang diganti


In [5]:
# kita lihat nilai - nilai unique dalam datanya
data_1['Hasil'].unique()

array(['Tidak ada yang diganti', 'Oli diganti',
       'Tali gas diganti kemudian oli',
       'Slongsong gas diganti kemudian tali gas kemudian oli',
       'Semua komponen diganti berurutan', 'Tali gas diganti',
       'Slongsong gas diganti', 'Butuh pengecekan lebih lanjut',
       'Slongsong gas diganti kemudian tali gas kemudian cek lebih lanjut'],
      dtype=object)

In [6]:
# kita hitung berapa jumlah kelas di kolom 'Hasil' tersbeut 
data_1['Hasil'].value_counts()

Tidak ada yang diganti                                               4
Tali gas diganti kemudian oli                                        1
Slongsong gas diganti                                                1
Tali gas diganti                                                     1
Butuh pengecekan lebih lanjut                                        1
Slongsong gas diganti kemudian tali gas kemudian oli                 1
Slongsong gas diganti kemudian tali gas kemudian cek lebih lanjut    1
Oli diganti                                                          1
Semua komponen diganti berurutan                                     1
Name: Hasil, dtype: int64

Terlihat bahwa jumlah class yang tersebar di dalam kolom Hasil itu bersifat tidak merata. artinya class 'Tidak ada yang diganti' memiliki jumlah paling banyak (mendominasi) dibandingkan dengan class yang lain seperti 'Oli diganti', 'tali gas diganti' dll. sehingga, agar menghasilkan model yang baik, maka jumlah class ini harus disama-ratakan (balancing) agar distribusi nilainya merata. sehingga akan semakin baik dalam menyimpulkan/memprediksi data

### Random oversampling

In [7]:
'''
import library imblearn. karena data kita mengalami imbalance class
di mana imbalance class ini merupakan kondisi ketidakseimabagan jumlah 
class yang terdistribusi di dalam suatu field/kolom, sehingga menyebabkan
sistem tersebut mengalami kesulitasn dalam training dan validasi.
'''
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# kita pecah datasetnya menjadi dua bagian yaitu variabel features dan variabel target
# X ==> berisi variabel-variabel features
X = data_1.iloc[:, :-1]

# Y ==> berisi sebuah variabel target
y = data_1.iloc[:, -1]

# mulai sampling
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(X, y)
print('dimensi setelah oversampling')
print('X_resampled\n', X_resampled.shape)
print('y_resampled\n', y_resampled.shape)

dimensi setelah oversampling
X_resampled
 (36, 4)
y_resampled
 (36,)


In [8]:
# lihat hasil setelah oversampling
out = pd.DataFrame({'x_resampled':y_resampled})
out['x_resampled'].value_counts()

Tali gas diganti kemudian oli                                        4
Slongsong gas diganti                                                4
Tidak ada yang diganti                                               4
Tali gas diganti                                                     4
Butuh pengecekan lebih lanjut                                        4
Slongsong gas diganti kemudian tali gas kemudian oli                 4
Slongsong gas diganti kemudian tali gas kemudian cek lebih lanjut    4
Oli diganti                                                          4
Semua komponen diganti berurutan                                     4
Name: x_resampled, dtype: int64

Trlihat bahwa jumlah setiap class yang tersebar di kolom hasil memiliki jumlah yang sama yakni sebanyak 4. hal ini berarti variabel target telah memiliki class class yang balance

In [9]:
# REKONSTRUKSI ULANG DATA_1-Nya
data_1_rev = pd.DataFrame(X_resampled)
data_1_rev['hasil'] = y_resampled
data_1_rev.head()

,0,1,2,3,hasil
0,Normal,Bagus,Bagus,<1bulan,Tidak ada yang diganti
1,Normal,Bagus,Bagus,>1bulan,Oli diganti
2,Normal,Bagus,Putus,>1bulan,Tali gas diganti kemudian oli
3,Normal,Berkarat,Putus,>1bulan,Slongsong gas diganti kemudian tali gas kemudi...
4,Berat,Berkarat,Putus,>1bulan,Semua komponen diganti berurutan


#### perbaiki nama kolomnya 

In [10]:
list_nama_kolom = data_1.columns # out ==> ['Gas', 'Slongsong Gas', 'Tali Gas', 'Oli', 'Hasil']
data_1_rev.columns = list_nama_kolom
data_1_rev

,Gas,Slongsong Gas,Tali Gas,Oli,Hasil
0,Normal,Bagus,Bagus,<1bulan,Tidak ada yang diganti
1,Normal,Bagus,Bagus,>1bulan,Oli diganti
2,Normal,Bagus,Putus,>1bulan,Tali gas diganti kemudian oli
3,Normal,Berkarat,Putus,>1bulan,Slongsong gas diganti kemudian tali gas kemudi...
4,Berat,Berkarat,Putus,>1bulan,Semua komponen diganti berurutan
5,Normal,Bagus,Putus,<1bulan,Tali gas diganti
6,Normal,Berkarat,Bagus,<1bulan,Slongsong gas diganti
7,Berat,Bagus,Bagus,<1bulan,Butuh pengecekan lebih lanjut
8,Berat,Berkarat,Putus,<1bulan,Slongsong gas diganti kemudian tali gas kemudi...
9,Normal,Bagus,Bagus,>1bulan,Tidak ada yang diganti


### Data Encoding (Transformation)

In [11]:
from sklearn import preprocessing
trans = preprocessing.LabelEncoder()
columns = data_1_rev.columns
data_encoded_storage = {}
labels_storage = {} # ini untuk meta data
for col in columns:
    out = trans.fit_transform(data_1_rev[col])
    out_lbl = trans.classes_
    labels_storage.update({col:out_lbl})
    data_encoded_storage.update({col:out})

### Rekonstruksi kembali dataset data_1_rev-nya ke dalam dataframe

In [12]:
data_1_rev = pd.DataFrame(data_encoded_storage)
data_1_rev.head() # lihat cuman 5 record data saja

,Gas,Slongsong Gas,Tali Gas,Oli,Hasil
0,1,0,0,0,8
1,1,0,0,1,1
2,1,0,1,1,7
3,2,1,1,1,5
4,0,1,1,1,2


### Splitting datanya menjadi training dan testing

In [13]:
from sklearn import model_selection
FEATURES = data_1_rev.iloc[:, :-1]
TARGET = data_1_rev.iloc[:, -1]
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    FEATURES, 
    TARGET, 
    test_size=0.2
)
# cek dimensi datanya 
print('x_train : ', x_train.shape)
print('y_train : ', y_train.shape)
print('x_test  : ',  x_test.shape)
print('y_test  : ',  y_test.shape)

x_train :  (28, 4)
y_train :  (28,)
x_test  :  (8, 4)
y_test  :  (8,)


In [14]:
# import library algoritma pelatihannya
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

# import library untuk mengukur akurasi dan laporan hasil klasifikasi
from sklearn.metrics import accuracy_score, classification_report

# inisialisasi model-nya
models = {
    "Support_vector_machine": SVC(kernel="linear", gamma="auto"),
    "Decission_Tree": DecisionTreeClassifier(criterion='entropy'),
    "multi_layer_perceptron": MLPClassifier(hidden_layer_sizes=(250),max_iter=1000,batch_size=100)
}

# membuat fungsi untuk melakukan fungsi fitting atau training
def train_model (model,xtrain,ytrain):
    model.fit(xtrain,ytrain)

# membuat fungsi untuk menampilkan akurasi dan classifiation_report
def predict(i,model,xtest,ytest):
    pred = model.predict(xtest)
    accuracy = accuracy_score(ytest, pred) * 100
    print("\n",i,":")
    print(classification_report(ytest, pred))
    print("accuracy : \n", round(accuracy,3),"%\n")

# melakukan training dengan menggunakan semua algoritma yang telah diinisialisasi
for i in models:
    train_model(models[i],x_train,y_train)
    predict(i,models[i],x_test,y_test)


 Support_vector_machine :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       1.00      1.00      1.00         1
           8       0.00      0.00      0.00         2

   micro avg       0.75      0.75      0.75         8
   macro avg       0.62      0.62      0.62         8
weighted avg       0.75      0.75      0.75         8

accuracy : 
 75.0 %


 Decission_Tree :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           3       0.50      1.00      0.67         1
           4       1.00      1.00      1.00         2
           

In [15]:
labels_storage.get('Hasil')

array(['Butuh pengecekan lebih lanjut', 'Oli diganti',
       'Semua komponen diganti berurutan', 'Slongsong gas diganti',
       'Slongsong gas diganti kemudian tali gas kemudian cek lebih lanjut',
       'Slongsong gas diganti kemudian tali gas kemudian oli',
       'Tali gas diganti', 'Tali gas diganti kemudian oli',
       'Tidak ada yang diganti'], dtype=object)

## Persiapan Production ke web service

In [18]:
# kita siimpan model yang sudah dihasilkan ke format pickle (.pkl)
# import library pickle-nya
import pickle
import os 

# ambil models nya
data_no = 1
folder_name = 'model_{}'.format(data_no)

if os.path.exists(folder_name):
    print('path exist')
else:
    os.mkdir(folder_name)

model_mlp = models['multi_layer_perceptron']

# simpan file model yang sudah dilatih ke dalam format pickle
model_name = '{}/model_data_{}.pkl'.format(folder_name, data_no)
pickle.dump(model_mlp, open(model_name, 'wb'))

# simpan variabel class_keputusan ke dalam format pickle
label_name = '{}/label_data_{}.pkl'.format(folder_name, data_no)
pickle.dump(labels_storage, open(label_name, 'wb'))

print('Done')

path exist
Done


In [21]:
# (75+100+100+80) / 4 